<a href="https://colab.research.google.com/github/zaynabreza/ThreatGator/blob/main/Model%20Training/FYP_Relation_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relation Extraction 

### Installations and Downloads

In [ ]:
!pip install -U spacy-nightly --pre

In [ ]:
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
!pip uninstall transformers

Found existing installation: transformers 4.16.2
Uninstalling transformers-4.16.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.16.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.16.2


In [ ]:
# !pip install transformers==4.15.0

  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.5
    Uninstalling tokenizers-0.11.5:
      Successfully uninstalled tokenizers-0.11.5


4.16.2


In [ ]:
!pip install -U spacy[transformers]

  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)


In [ ]:
%cd rel_component

/content/rel_component


In [ ]:
!mkdir data

### Training of RE Model

In [ ]:
!spacy project run train_gpu # command to train train transformers
!spacy project run evaluate # command to evaluate on test dataset


================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_trf.cfg --output training --paths.train data/train.spacy --paths.dev data/dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-02-22 15:47:38,077] [INFO] Set up nlp object from config
[2022-02-22 15:47:38,088] [INFO] Pipeline: ['transformer', 'relation_extractor']
[2022-02-22 15:47:38,092] [INFO] Created vocabulary
[2022-02-22 15:47:38,094] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 382kB/s]
Downloading: 100% 878k/878k [00:01<00:00, 827kB/s] 
Downloading: 100% 446k/446k [00:00<00:00, 505kB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 1.48MB/s]
Downloading: 100% 478M/478M [00:10<00:00, 45.7MB/s]
Some weights of the model checkpoint at roberta-base were not used when initial

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r training /content/gdrive/MyDrive/FYP

In [ ]:
f = open('relations-test.json')
 
# returns JSON object as
# a dictionary
JSON_file = json.load(f)
print(JSON_file)
with open('test.jsonl', 'w') as outfile:
    for entry in JSON_file:
        json.dump(entry, outfile)
        outfile.write('\n')

[{'documentName': 'sentences-test_2022-02-20_10:45:50.2.txt', 'document': '\n Charges filed relate to Lazarus Group  s long - running cyber crime spree .\n The trio from APT38 also attempted to steal over $ 1 . 2bn from banks in Africa , Bangladesh , Malta , Mexico and Vietnam .\n Lazarus are further implicated in the creation of the destructive WannaCry 2 . 0 ransomware in May 2017 .\n Lazarus Group uses multiple proxies to obfuscate network traffic from victims .\n Researchers investigated a widespread and opportunistic WCry ( also known as WanaCry , WanaCrypt , and Wana Decrypt0r ) ransomware made by lazarus group .\n In late 2017 , HIDDEN COBRA used KillDisk , a disk - wiping tool , in an attack against an online casino based in Central America .\n HIDDEN COBRA stole tens of millions of dollars  worth of cryptocurrency .\n The campaign  s use of an SMB worm to distribute WCry contributed to the ransomware  s virulence .\n Lazarus Group malware SHARPKNOT overwrites and deletes the M

### Joint Entity and Relation Extraction Pipeline:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


####Testing and Loading NER

In [ ]:
import spacy
nlp = spacy.load("/content/gdrive/MyDrive/FYP/NERSpacy/model-best")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
import os
os.getcwd()

'/content/rel_component'

In [ ]:
def getMax(x):
  return max(x, key=x.get)

####Example 1

In [ ]:
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
import os
os.chdir("scripts")
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors# We load the relation extraction (REL) model
%cd ..
nlp2 = spacy.load("/content/gdrive/MyDrive/FYP/training/model-best")# We take the entities generated from the NER pipeline and input them to the REL pipeline
# print(nlp2)
Text=["Lazarus Group targeted Pakistani company Telenor using KillDisk"]
print("*****Sentence*****")
print(Text)
print("*****Entities*****")
for doc in nlp.pipe(Text, disable=["tagger"]):
   print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")
nlp2.add_pipe('sentencizer')
print("*****Relationships*****")
for name, proc in nlp2.pipeline:
  doc = proc(doc) # Here, we split the paragraph into sentences and apply the relation extraction for each pair of entities found in each sentence.
printed=[]
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                # max(stats, key=stats.get)
                for relation in rel_dict:
                  if (e.text, b.text) not in printed and (b.text, e.text) not in printed:
                    print(f" entities: {e.text, b.text} --> predicted relation: {getMax(rel_dict)}")
                    printed.append((e.text, b.text))
                # if rel_dict['TARGETS'] >=0.9 :
                #   print(f" entities: {e.text, b.text} --> predicted relation: {getMax(rel_dict)}")
                # if rel_dict['USES'] >=0.9 :
                #   print(f" entities: {e.text, b.text} --> predicted relation: {getMax(rel_dict)}")
                # if rel_dict['LOCATED-AT'] >=0.9 :
                #   print(f" entities: {e.text, b.text} --> predicted relation: {getMax(rel_dict)}")

/content/rel_component


/usr/local/lib/python3.7/dist-packages/spacy_transformers/pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


*****Sentence*****
['Lazarus Group targeted Pakistani company Telenor using KillDisk']
*****Entities*****
spans: [(0, 'Lazarus Group', 'TA'), (3, 'Pakistani', 'L'), (5, 'Telenor', 'ID'), (7, 'KillDisk', 'T')]
*****Relationships*****
 entities: ('Lazarus Group', 'Pakistani') --> predicted relation: TARGETS
 entities: ('Lazarus Group', 'Telenor') --> predicted relation: TARGETS
 entities: ('Lazarus Group', 'KillDisk') --> predicted relation: USES
 entities: ('Pakistani', 'Telenor') --> predicted relation: LOCATED-AT
 entities: ('Pakistani', 'KillDisk') --> predicted relation: USES
 entities: ('Telenor', 'KillDisk') --> predicted relation: USES


####Example 2









In [ ]:
print("*****Sentence*****")
print(Text)
Text=["Iranian hackers target VMware Horizon servers with Log4j exploits"]
print("*****Entities*****")
for doc in nlp.pipe(Text, disable=["tagger"]):
   print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")

print("*****Relationships*****")
nlp2 = spacy.load("/content/gdrive/MyDrive/FYP/training/model-best")# We take the entities generated from the NER pipeline and input them to the REL pipeline
print(nlp2)
nlp2.add_pipe('sentencizer')
for name, proc in nlp2.pipeline:
  doc = proc(doc) # Here, we split the paragraph into sentences and apply the relation extraction for each pair of entities found in each sentence.
printed=[]
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                # max(stats, key=stats.get)
                for relation in rel_dict:
                  if (e.text, b.text) not in printed and (b.text, e.text) not in printed:
                    print(f" entities: {e.text, b.text} --> predicted relation: {getMax(rel_dict)}")
                    printed.append((e.text, b.text))

*****Sentence*****
['Iranian hackers target VMware Horizon servers with Log4j exploits']
*****Entities*****
spans: [(0, 'Iranian', 'L'), (3, 'VMware', 'ID'), (4, 'Horizon servers', 'V'), (7, 'Log4j exploits', 'M')]
*****Relationships*****


/usr/local/lib/python3.7/dist-packages/spacy_transformers/pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


 entities: ('Iranian', 'VMware') --> predicted relation: TARGETS
 entities: ('Iranian', 'Horizon servers') --> predicted relation: TARGETS
 entities: ('Iranian', 'Log4j exploits') --> predicted relation: USES
 entities: ('VMware', 'Horizon servers') --> predicted relation: HAS
 entities: ('VMware', 'Log4j exploits') --> predicted relation: HAS
 entities: ('Horizon servers', 'Log4j exploits') --> predicted relation: DELIVERS
